In [82]:
import numpy as np
from numpy import *
import math
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.pyplot import cm

import cv2

   

%matplotlib inline



def draw(pointList, makeVideo=False):
       
    
    fig, ax = plt.subplots()

    plt.ylim(0.0, 100.0)
    plt.xlim(0.0, 100.0)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.grid(linestyle='--')


    #--------------------draw pockets--------------------    
    alphaValue = 0.4
    for pt in corners:
        ax.add_artist(plt.Circle(pt, epsilon1, color='orange', alpha=alphaValue))
        #make the fallPocket red
    ax.add_artist(plt.Circle((sideLen/2, sideLen/2), epsilon2, color='orange', alpha=alphaValue))
    
    #make the fallPocket red
    if fallPocket==(sideLen/2, sideLen/2):
        ax.add_artist(plt.Circle(fallPocket, epsilon2, color='red', alpha=alphaValue))
    else:
        ax.add_artist(plt.Circle(fallPocket, epsilon1, color='red', alpha=alphaValue))
    #--------------------draw trajectory---------------
    xList = []
    yList = []
    for pt in pointList:
        xList.append(pt[0])
        yList.append(pt[1])
    
    colors = cm.rainbow(np.linspace(0, 1, len(xList)))
    
    images=[]
    for i in range(len(xList)-1):
        plt.plot(xList[i:i+2], yList[i:i+2], color=colors[i], marker='o')
        imgName = 'images/' + str(i) + '.png'
        plt.savefig(imgName)
        images.append(cv2.imread(imgName))
    
    plt.title("Trajectroy: p0=" + str(pointList[0]) + ", p1=" + str(pointList[1]))
    plt.xlabel("x")
    plt.ylabel("y")
    plt.show()
    
    
    print("len(images) =", len(images))
    
    
    #--------------------now make video---------------
    if makeVideo:
        
        height,width,layers=images[1].shape
        video=cv2.VideoWriter('billiard.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 5, (width,height))

        for j in range(len(images)):
            video.write(images[j])

        cv2.destroyAllWindows()
        video.release()

        print("Video Done!")

    
    
    
   
  

In [83]:
#sides = 0, 1, 2, 3 (bottom, right, top, left)
sideLen = 100
corners = [(0,0), (sideLen, 0), (sideLen, sideLen), (0, sideLen)]

#given a side s, return its endpoints (x1 ,y1, x2, y2)

sideCoords = []
sideCoords.append([0,0, sideLen, 0])#bottom side
sideCoords.append([sideLen,0, sideLen, sideLen])#right side
sideCoords.append([sideLen,sideLen, 0, sideLen])#top side
sideCoords.append([0,sideLen, 0, 0])#left side

    

In [84]:
def getSide(pt):
    x,y = pt
    if x==0:
        return 3
    if x==sideLen:
        return 1
    if y==0:
        return 0
    if y==sideLen:
        return 2

In [85]:
# input: point_i (pi), point_i+1 (pil), line_i (li), side_i (si)
# output: point_i+1 (piNew), point_i+2 (piNextNew), line_i+1 (liNew), side_i (siNew)

#siNext is the side of piNext

def getNextPoint(pi, piNext):

    temp = [sideLen, sideLen, 0, 0]
    siNext = getSide(piNext)
#     -------------------------------------------------------------------------------
    if siNext%2==1: #1, or 3
            
        newy = 0
        if piNext[1] > pi[1]:
            newy = sideLen # axis = horizontal,    expect newy=sideLen or 0
#         print("expect newy = ", newy)
        
        
        if siNext==1:
            newx = piNext[0] -  abs( (piNext[0]-pi[0]) * (newy-piNext[1]) / (piNext[1]-pi[1]) )
        else:#siNext==3
            newx = piNext[0]  +  abs( (piNext[0]-pi[0]) * (newy-piNext[1]) / (piNext[1]-pi[1]) )
            
#         print("tan = ", abs( (piNext[0]-pi[0]) / (piNext[1]-pi[1]) ))
#         print("temp newx =", newx)

        if not (0 <= newx and newx <= sideLen):  #si increments two
#             print("increment two")
            
            newx = 0
            if piNext[0] < pi[0]:
                newx = sideLen
            
            if piNext[1] > pi[1]:
                newy = piNext[1]    +   abs((piNext[1]-pi[1]) * (piNext[0] - newx) /  (piNext[0] - pi[0]))
            else:
                newy = piNext[1]    -   abs((piNext[1]-pi[1]) * (piNext[0] - newx) /  (piNext[0] - pi[0]))
    
    
#     -------------------------------------------------------------------------------
    if siNext%2==0: #2, or 0(4)

            
        newx = 0
        if piNext[0] > pi[0]:
            newx = sideLen # axis = vertical,    expect newy=sideLen or 0
        
#         print("expect newx =", newx)
        
        if siNext==2:
            newy = piNext[1] -   abs( (piNext[1]-pi[1]) * (newx-piNext[0]) / (piNext[0]-pi[0]) )
#             print("expect newy =", newy)
        else:#siNext==0
            newy = piNext[1] +   abs( (piNext[1]-pi[1]) * (newx-piNext[0]) / (piNext[0]-pi[0]) )
    

    
        if not (0 <= newy and newy <= sideLen):  #si increments two
#             print("increment two")
            
            newy = 0
            if piNext[1] < pi[1]:
                newy = sideLen
            
            if piNext[0] > pi[0]:
                newx = piNext[0]    +  abs((piNext[0]-pi[0]) * (piNext[1] - newy) /  (piNext[1] - pi[1]))
            else:
                newx = piNext[0]    -  abs((piNext[0]-pi[0]) * (piNext[1] - newy) /  (piNext[1] - pi[1]))
    

        
    return (newx, newy)


In [86]:

def pointDist(pt1, pt2):
    x1, y1 = pt1
    x2, y2 = pt2
    return np.sqrt((x1-x2)**2 + (y1-y2)**2 )
    
#distance between ptC and the line segment: (ptA to ptB)
def dist(A, B, C):
    x1, y1 = A
    x2, y2 = B
    x3, y3 = C
    
    a = pointDist(B, C)
    b = pointDist(A, C)
    c = pointDist(A, B)
    
    cosA = (b**2+c**2-a**2) / (2*b*c)
    cosB = (a**2+c**2-b**2) / (2*a*c)
    K = (y2-y1) / (x2-x1)
    B = y1-K*x1
    if cosA > 0 and cosB > 0:# both angles A and B < 90
        return abs(y3 - K*x3 - B) / np.sqrt(1**2 + (-K)**2)
    else:
        return min(a, b)
    

def fall(pt1, pt2):
    global fallPocket
    for pt in corners:
        if dist(pt1, pt2, pt) < epsilon1:
#             print("Fall into pocket: ", pt)
            fallPocket = pt
            return True
    centerPt = (sideLen/2, sideLen/2)
    if dist(pt1, pt2, centerPt) < epsilon2:
        fallPocket = centerPt
#         print("Fall into CENTER pocket: ", centerPt)
        return True
    
    return False

In [87]:
def billiard(p0, p1, ptList):
    global fallPocket
    
    if dist(p0, p1, (sideLen/2, sideLen/2)) < epsilon2:
        fallPocket = (sideLen/2, sideLen/2)
        return
    for pt in corners:
        if pt==(0,0):
            continue
        if dist(p0, p1, pt) < epsilon1:
            fallPocket = pt
            return
    
    
    pi = p0
    piNext = p1
    
    count = 0
    while(True):
#         print("pi =", pi, "  piNext =", piNext)
        
        
        count+=1
        newx, newy = getNextPoint(pi, piNext)
        
        pi = piNext
        piNext = (newx, newy)
        
        ptList.append(piNext)
        if fall(pi, piNext):
            break
#         if count > 100:
#             print("COUNT OF LINE SEGMENTS > 100, WE STOP!")
#             break
        

# You could try two things:

## 1. make epsilon2 = epslon1, see if the probability raio (prob of corners / prob of center) is fixed
## 2. make the prob ratio fixed (=1) and see ratio of epsilon2/epsilon1

## Try 1

In [88]:
def conjecture(p0, p1, drawImg=False, drawWithVideo=False):
    ptList = [p0, p1]
    global fallPocket
    fallPocket = (999, 999)

    billiard(p0, p1, ptList)
    if drawImg:
        draw(ptList, makeVideo=drawWithVideo)
    return fallPocket

In [89]:
import math
p1Samples = []


num = 1000
for angle in np.linspace(1, 45, num): #1~45 degree
    theta = math.radians(angle)
    tan = math.tan(theta)
    p1Samples.append((sideLen, sideLen*tan))

for angle in np.linspace(45, 90, num): #45~90 degree
    theta = math.radians(angle)
    tan = math.tan(theta)
    p1Samples.append((sideLen/tan, sideLen))


    
print("len(p1Samples) = ", len(p1Samples))

len(p1Samples) =  2000


In [90]:
from colorama import Fore, Back, Style 
print(Fore.RED + 'some red text') 
print(Style.RESET_ALL) 
print('back to normal now') 

some red text

back to normal now


In [99]:
for epsilon1 in np.linspace(2, sideLen*np.sqrt(2)/4, 100):
    epsilon2 = epsilon1*0.85
    p0 = (0,0)

    ct1 = 0 #count falls into corner pockets
    ct2 = 0 #count falls into center pocket


    for p1 in p1Samples:
        fallPocket = conjecture(p0, p1, drawImg=False, drawWithVideo=False)
        if fallPocket==(sideLen/2, sideLen/2):
            ct2 +=1
        else:
            ct1 += 1


    print("epsilon2, epsilon1, ct1, ct2 =", epsilon2, epsilon1, ct1, ct2,)
    print(Fore.RED+ "ct1/ct2 =", ct1/ct2)
    print(Style.RESET_ALL, end="") 
    

epsilon2, epsilon1, ct1, ct2 = 1.7 2.0 759 1241
ct1/ct2 = 0.6116035455278002
epsilon2, epsilon1, ct1, ct2 = 1.9863842242467502 2.3369226167608828 775 1225
ct1/ct2 = 0.6326530612244898
epsilon2, epsilon1, ct1, ct2 = 2.2727684484935002 2.673845233521765 762 1238
ct1/ct2 = 0.6155088852988692
epsilon2, epsilon1, ct1, ct2 = 2.5591526727402507 3.010767850282648 764 1236
ct1/ct2 = 0.6181229773462783
epsilon2, epsilon1, ct1, ct2 = 2.8455368969870007 3.34769046704353 768 1232
ct1/ct2 = 0.6233766233766234
epsilon2, epsilon1, ct1, ct2 = 3.1319211212337508 3.684613083804413 764 1236
ct1/ct2 = 0.6181229773462783
epsilon2, epsilon1, ct1, ct2 = 3.4183053454805012 4.021535700565296 765 1235
ct1/ct2 = 0.6194331983805668
epsilon2, epsilon1, ct1, ct2 = 3.7046895697272517 4.358458317326178 771 1229
ct1/ct2 = 0.6273393002441009
epsilon2, epsilon1, ct1, ct2 = 3.9910737939740013 4.69538093408706 772 1228
ct1/ct2 = 0.6286644951140065
epsilon2, epsilon1, ct1, ct2 = 4.277458018220752 5.032303550847944 773 1227


epsilon2, epsilon1, ct1, ct2 = 22.319664145766016 26.258428406783548 947 1053
ct1/ct2 = 0.899335232668566
epsilon2, epsilon1, ct1, ct2 = 22.60604837001276 26.595351023544428 950 1050
ct1/ct2 = 0.9047619047619048
epsilon2, epsilon1, ct1, ct2 = 22.892432594259514 26.93227364030531 952 1048
ct1/ct2 = 0.9083969465648855
epsilon2, epsilon1, ct1, ct2 = 23.178816818506267 27.269196257066195 953 1047
ct1/ct2 = 0.9102196752626552
epsilon2, epsilon1, ct1, ct2 = 23.465201042753012 27.606118873827075 954 1046
ct1/ct2 = 0.9120458891013384
epsilon2, epsilon1, ct1, ct2 = 23.751585266999765 27.94304149058796 957 1043
ct1/ct2 = 0.9175455417066155
epsilon2, epsilon1, ct1, ct2 = 24.037969491246514 28.279964107348842 958 1042
ct1/ct2 = 0.9193857965451055
epsilon2, epsilon1, ct1, ct2 = 24.324353715493267 28.616886724109726 960 1040
ct1/ct2 = 0.9230769230769231
epsilon2, epsilon1, ct1, ct2 = 24.610737939740016 28.953809340870606 961 1039
ct1/ct2 = 0.9249278152069298
epsilon2, epsilon1, ct1, ct2 = 24.8971221

In [ ]:
# for epsilon2 in np.linspace(0, 2, 30)[1:]:
#     epsilon1 = epsilon2
#     p0 = (0,0)

#     ct1 = 0 #count falls into corner pockets
#     ct2 = 0 #count falls into center pocket


#     for p1 in p1Samples:
#         fallPocket = conjecture(p0, p1, drawImg=False, drawWithVideo=False)
#         if fallPocket==(sideLen/2, sideLen/2):
#             ct2 +=1
#         else:
#             ct1 += 1


#     print("epsilon2, epsilon1, ct1, ct2 =", epsilon2, epsilon1, ct1, ct2,)
#     print(Fore.RED+ "ct1/ct2 =", ct1/ct2)
#     print(Style.RESET_ALL, end="") 
    

## Try 2

In [98]:
epsilon2 = 5
epsilon1 = epsilon2*1.66
p0 = (0,0)

ct1 = 0 #count falls into corner pockets
ct2 = 0 #count falls into center pocket


for p1 in p1Samples:
    fallPocket = conjecture(p0, p1, drawImg=False, drawWithVideo=False)
    if fallPocket==(sideLen/2, sideLen/2):
        ct2 +=1
    else:
        ct1 += 1


print("epsilon2, epsilon1, ct1, ct2 =", epsilon2, epsilon1, ct1, ct2,)
print(Fore.RED+ "ct1/ct2 =", ct1/ct2)
print(Style.RESET_ALL, end="") 
    

epsilon2, epsilon1, ct1, ct2 = 5 8.299999999999999 1034 966
ct1/ct2 = 1.0703933747412009


In [96]:
epsilon2 = 1
epsilon1 = epsilon2*1.66
p0 = (0,0)

ct1 = 0 #count falls into corner pockets
ct2 = 0 #count falls into center pocket


for p1 in p1Samples:
    fallPocket = conjecture(p0, p1, drawImg=False, drawWithVideo=False)
    if fallPocket==(sideLen/2, sideLen/2):
        ct2 +=1
    else:
        ct1 += 1


print("epsilon2, epsilon1, ct1, ct2 =", epsilon2, epsilon1, ct1, ct2,)
print(Fore.RED+ "ct1/ct2 =", ct1/ct2)
print(Style.RESET_ALL, end="") 
    

epsilon2, epsilon1, ct1, ct2 = 1 1.66 997 1003
ct1/ct2 = 0.9940179461615155


In [97]:
epsilon2 = 0.5
epsilon1 = epsilon2*1.66
p0 = (0,0)

ct1 = 0 #count falls into corner pockets
ct2 = 0 #count falls into center pocket


for p1 in p1Samples:
    fallPocket = conjecture(p0, p1, drawImg=False, drawWithVideo=False)
    if fallPocket==(sideLen/2, sideLen/2):
        ct2 +=1
    else:
        ct1 += 1


print("epsilon2, epsilon1, ct1, ct2 =", epsilon2, epsilon1, ct1, ct2,)
print(Fore.RED+ "ct1/ct2 =", ct1/ct2)
print(Style.RESET_ALL, end="") 
    

epsilon2, epsilon1, ct1, ct2 = 0.5 0.83 1000 1000
ct1/ct2 = 1.0


## Look at the previous setting where we do not have center pocket

In [95]:
epsilon2 = 0
epsilon1 = 1
p0 = (0,0)

ct0 = 0 #count falls into left bottom pocket - should be 0
ct1 = 0 #count falls into right bottom pocket
ct2 = 0 #count falls into right up pocket
ct3 = 0 #count falls into left up  pocket



for idx, p1 in enumerate(p1Samples):
    if idx%100 ==0:
        print("idx =", idx)
    fallPocket = conjecture(p0, p1, drawImg=False, drawWithVideo=False)
    if fallPocket==(0,0):
        ct0 += 1
    elif fallPocket==(sideLen,0):
        ct1 += 1
    elif fallPocket==(sideLen,sideLen):
        ct2 += 1
    elif fallPocket==(0,sideLen):
        ct3 += 1
    else:
        print("******ERROR********* fallPocket =", fallPocket)
    


print("epsilon1, ct0, ct1, ct2, ct3 =", epsilon1, ct0, ct1, ct2, ct3) 
total = ct0+ct1+ct2+ct3
print(Fore.RED+ "normalized  ct0, ct1, ct2, ct3=",  ct0/total, ct1/total, ct2/total, ct3/total)
print(Style.RESET_ALL, end="") 

idx = 0
idx = 100
idx = 200
idx = 300
idx = 400
idx = 500
idx = 600
idx = 700
idx = 800
idx = 900
idx = 1000
idx = 1100
idx = 1200
idx = 1300
idx = 1400
idx = 1500
idx = 1600
idx = 1700
idx = 1800
idx = 1900
epsilon1, ct0, ct1, ct2, ct3 = 1 0 654 673 673
normalized  ct0, ct1, ct2, ct3= 0.0 0.327 0.3365 0.3365
